# Plot for Summary Report: PCA and K-Means Clustering

In [1]:
%run 'helpers-for-report-plot-generation.ipynb'
from datetime import datetime

In [ ]:
def get_raw_data():
    """Get raw data from local database"""
    milk = get_dataset('milk_volume')
    birthdates = get_dataset('birthdates')
    genetics = get_dataset('genetics')
    classification = get_dataset('classification')
        
    return milk, birthdates, genetics, classification

def transform_milk(data):
    """Transform milk features"""
    features = ['animal_id','date','milk_weight']
    return data[features]

def transform_genetics(data):
    """Transform and select genetics features"""
    data = data.drop(axis=1, labels=['name'])
    print('\nCurrently Available Genetics Features: \n', data.columns.values)
    features = ['animal_id', 'milk', 'ctpi', 'udc']
    print('\nSelected Genetics Features: \n', features)
    data = data.drop_duplicates(subset=['animal_id'], keep='last')
    return data.fillna(data.mean())[features]

def transform_classification(data):
    """Transform and select classification features"""
    data = data.drop(axis=1, labels=['category', 'age', 'date_calved', 'date'])
    print('\nCurrently Available Classification Features: \n', data.columns.values)
    features = ['animal_id', 'milk', 'ctpi', 'udc']
    print('\nSelected Classification Features: \n', features)
    features = ['animal_id', 'final_score', 'breed_age_average']
    data = data.drop_duplicates(subset=['animal_id'], keep='last')
    return data.fillna(data.mean())[features]

def number_of_days_in_period(start, end):
    """Gets number of days between two string dates"""
    end = datetime.strptime(end, '%m-%d-%Y')
    start = datetime.strptime(start, '%m-%d-%Y')
    return (start - end).days

def build_performance_profiles_for_period(milk, birthdates, start='1-1-2016', end='12-31-2017'):
    """Build summary performance profiles for date range"""
    period_milk = milk.set_index('date')[start:end]
    period_milk = pd.merge(left=period_milk, right=birthdates, left_on='animal_id', right_on='animal_id')
    
    days_in_period = number_of_days_in_period(start, end)
    
    profiles = []
    for animal_id in period_milk['animal_id'].unique():
        animal_milk_for_period = period_milk[period_milk['animal_id'] ==  animal_id]
        
        profile = {}
        profile['animal_id'] = animal_id 
        profile['days_milked'] = animal_milk_for_period.count()['animal_id']
        profile['total_milkweight'] = animal_milk_for_period['milk_weight'].sum()
        profile['daily_milkweight'] = round(animal_milk_for_period['milk_weight'].mean(), 2)
        
        birthdate = pd.to_datetime(animal_milk_for_period['birthdate'].values[0]).strftime('%m-%d-%Y')
        
        profile['days_aged'] = number_of_days_in_period(end, birthdate)
        
        profiles.append(profile)
        
    data = pd.DataFrame(profiles)
    data = data.sort_values('daily_milkweight')
    return data

def merge_data(performance_profiles, genetics, classification):
    """Merge datasets performance_profiles, genetics, classifications into single dataset"""
    data = pd.merge(left=performance_profiles, right=genetics, left_on='animal_id', right_on='animal_id')
    data = pd.merge(left=data, right=classification, left_on='animal_id', right_on='animal_id')
    return data

def get_dataset_for_analysis():
    """Get transformed dataset for PCA analysis"""
    milk, birthdates, genetics, classification = get_raw_data()
    
    milk = transform_milk(milk)

    performance_profiles = build_performance_profiles_for_period(milk, birthdates)
    
    genetics = transform_genetics(genetics)
    classification = transform_classification(classification)
    
    return merge_data(performance_profiles, genetics, classification)

In [ ]:
data = get_dataset_for_analysis().sort_values('total_milkweight')
data = data[data['days_milked'] > 350]

In [ ]:
# Get top and bottom percentages of the herd
count = data.count()['animal_id']
percentage = 0.1

top_ids = data.head(int(count * percentage))['animal_id']
bot_ids = data.tail(int(count * percentage))['animal_id']

In [ ]:
# Scale data for PCA analysis
from sklearn.preprocessing import StandardScaler

animal_ids = data['animal_id']

data = data.drop(axis=1, labels=['animal_id'])
scaled_data = StandardScaler().fit_transform(data)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

def conduct_silhouette_average_analysis(data, k_max=11, title='title'):
    k_values = [x for x in range(2, k_max)]
    silhouette_averages = []
    for k in k_values:
        clusterer = KMeans(n_clusters=k, random_state=45)
        cluster_labels = clusterer.fit_predict(data)
        silhouette_averages.append(silhouette_score(data, cluster_labels))

    ax = plt.plot(k_values, silhouette_averages)
    plt.xlabel('Number of Clusters')
    plt.ylabel('Silhouette Average')
    plt.title(title)
    save_plot_for_summary_report('silhouette-score.png')
    plt.show()

In [ ]:
conduct_silhouette_average_analysis(scaled_data, k_max=11, title="Sum-of-Squares for k-clusters")

In [ ]:
from sklearn.decomposition import PCA

def conduct_pca_and_kmeans_analysis(data, animal_ids, pca_components=2, n_clusters=5):
    random_state = 25
    pca_components = PCA(n_components=pca_components, random_state=random_state).fit_transform(data)
    cluster_labels = KMeans(n_clusters=n_clusters, random_state=random_state).fit_predict(data)

    x = pca_components[:,0]
    y = pca_components[:,1]

    results = pd.DataFrame({'cluster_id':cluster_labels, 'x':x, 'y':y, 'animal_id':animal_ids})

    sns.lmplot('x', 'y', data=results, hue='cluster_id', fit_reg=False)
    _ = plt.title(f'Animal Clusters: k-means (k={n_clusters})')
    _ = plt.xlabel("")
    _ = plt.ylabel("")
    save_plot_for_summary_report('kmeans-clusters.png')
    plt.show()
    
    return results

In [ ]:
cluster_assignments = conduct_pca_and_kmeans_analysis(scaled_data, animal_ids, pca_components=2, n_clusters=6)

In [ ]:
def generate_profile_results(assignments, top_ids=[], bottom_ids=[]):
    cluster_results = []
    for cluster_id in assignments['cluster_id'].unique():
        cluster_assignments = assignments[assignments['cluster_id'] == cluster_id]
        
        results = {}
        results['cluster_id'] = cluster_id
        results['count_total'] = cluster_assignments['animal_id'].count()
        results['top_total'] = cluster_assignments[cluster_assignments['animal_id'].isin(top_ids)].count()['animal_id']
        results['bot_total'] = cluster_assignments[cluster_assignments['animal_id'].isin(bottom_ids)].count()['animal_id']
        results['bot_percent'] = results['bot_total'] / results['count_total']
        results['top_percent'] = results['top_total'] / results['count_total']
        
        cluster_results.append(results)
        
    columns = ['cluster_id', 'count_total', 'top_total', 'bot_total', 'top_percent', 'bot_percent']
    return pd.DataFrame(cluster_results)[columns].sort_values('cluster_id').set_index('cluster_id')

generate_profile_results(cluster_assignments, top_ids, bot_ids)